# Offshore Wind to Hydrogen Supply Chain

## Multi-Facility - Multi-Demand Model


Importing Necessary Modules

In [ ]:
from pyomo.core import *
from pyomo.environ import *
import numpy as np
import pandas as pd
import math
import time
import matplotlib.pyplot as plt

Parameters & Sets used in Model:

In [ ]:
# Parameters

Times = list(np.arange(0,365))

L = 100*0.621371 #Length of Pipeline in miles from Offshore facility to Onshore

Rho = 42.4 #Density of H2 in kg/m^3 at 700 Bar
V_max = 35 #Maximum Velocity of H2 in Pipeline in m/s

deltaP_l = 0.00839*1e6 #Pressure Drop per mile of Pipeline Pa/km


#WindTurbine Costs

PCWT =  3 #Annualized Capital Cost of 15MW Wind Turbine (M$)
POWT =  1.05 #Annual Maintenance Cost of 15MW Wind Turbine (M$)

#Desalination Costs

PCDes =  47*10**(-6)*1.35 #Annualized Capital Cost of Desalination per kt of h2 produced (M$)
PODes =  11.48*10**(-6)*1.35 #Annual Maintenance Cost of Desalination per kt of h2 produced (M$)

#Electrolyzer Costs

PCEL =  45.32*10**(0) #Annualized Capital Cost of Electrolyzer per GW of Capacity (M$)
POEL =  22.250*10**(0) #Annual Maintenance Cost of Desalination per GW of Capacity (M$)

#Pipeline Costs

PCP1 =  7061*L*1e-6 #Annualized Capital Cost of Pipelines in M$ for L in miles
PCP2 =  36*L*1e-6   #Annualized Capital Cost of Pipelines in M$ for L in miles
PCP3 =  7694*L*1e-6 #Annualized Capital Cost of Pipelines in M$ for L in miles
PCP4 =  34021*L*1e-6    #Annualized Capital Cost of CPipelines in M$ for L in miles

#Storage Costs

PCS =  1.56*248 #Annualized Capital Cost of Storage per kt of h2 produced (M$)
POS =  1.56*2.5/100*248 #Annual Maintenance Cost of Storage per kt of h2 produced (M$)

#Compression Costs

PCC =  1.55*82*(1e-3)*(1/1000)**0.6038 #Annualized Capital Cost of compression per GW of Compression Capacity (M$)
POC =  1.55*5/100*82*(1e-3)*(1/1000)**0.6038 #Annual Maintenance Cost of compression per GW of Compression Capacity (M$)

#Transmission Pump Costs

PCT =  1.55*82*(1e-3)*(1/1000)**0.8335 #Annualized Capital Cost of compression per GW of Transmisson Capacity (M$)
POT =  1.55*5/100*82*(1e-3)*(1/1000)**0.8335 #Annual Maintenance Cost of compression per GW of Transmisson Capacity (M$)

# Floating Platform 

FP_Cost = (500/9.8)*(0.14)  #Annualized Capital Cost of Floating Platform (M$) per GW of Electrolyzer Capacity

In [ ]:
def Optimal_Design(hub,states,buoys,Factories,Demands,DemandValue,RTTFile):
    
    print('Start of New Case For:')
    
    print('Hub Name:',hub)
    print('List of States:',states)
    print('List of Buoys:',buoys)
    print('List of Factories:',Factories)
    print('List of Demands:',Demands)
    print('List of DemandValues:',DemandValue)
    print('Name of the RTT File:', RTTFile)

    filename = str(RTTFile)

    RTT = np.array(pd.read_excel(filename,header=None))

    CF =[]

    for buoy in buoys:
        CFi = pd.read_csv('CF_Data/'+str(buoy)+'.csv')
        CFii = (CFi['CF']).values.tolist()
        CF.append(CFii)
    
    Times = list(np.arange(0,365))
    
    print('------------------')
    
    ##variable Definitions
    model = ConcreteModel()

    model.selector = Var(Factories,initialize=0,within=Binary) #Facility Selection

    #Decision Variables

    model.NWT = Var(Factories,initialize=1,within=Integers,bounds=(0,10000000)) #No of Wind Turbines to be installed
    model.CDes = Var(Factories,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #Desalination Capacity to be installed in H2 kt/hr
    model.CEL = Var(Factories,initialize=11,within=NonNegativeReals,bounds=(0,10**6)) #Electrolyzer Capacity to be installed in MW

    model.Cstore = Var(Factories,initialize=1,within=NonNegativeReals,bounds=(0.0,10**6)) #LH2 Storage Capacity to be installed in kTonnes

    model.CSA = Var(Factories,Demands,initialize=1,within=NonNegativeReals,bounds=(0.0,10**6)) #Pipeline Cross Sectional Area to be installed in inches^2


    model.CCompressor = Var(Factories,initialize=1,within=NonNegativeReals,bounds=(0.0,10**6)) #Compressor Capacity to be installed in ?
    model.CPump = Var(Factories,Demands,initialize=1,within=NonNegativeReals,bounds=(0.0,10**6)) #Pump Capacity to be installed in ?

    #PowerFlow Variables
    model.EWT = Var(Factories,Times,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #Power generated by Wind Turbines in each t
    model.EDes = Var(Factories,Times,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #Power consumed by Desalination in each t
    model.EEL = Var(Factories,Times,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #Power consumed by Electrolysis in each t
    model.ECompressor = Var(Factories,Times,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #Power consumed by Compression in each t
    model.EPump = Var(Factories,Demands,Times,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #Power consumed by Pumping in each t

    #Hydrogen Flow/Level Variables
    model.Lstore = Var(Factories,Times,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #LH2 Level in the tank in each t in ktonnes
    model.F = Var(Factories,Times,initialize=50,within=NonNegativeReals,bounds=(0,10**6)) #H2 produced in each t in ktonnes/hr
    model.M = Var(Factories,Demands,Times,initialize=0,within=NonNegativeReals,bounds=(0,10**6)) #H2 mass flow in pipeline each t in ktonnes/hr

    ## Piece-Wise Linear Approxiamtion:
    A_CSA1 =[] 
    B_T1 = []
    A_CSA3 = []
    B_T3 = []
    A_Comp = []
    B_Comp = []
    A_Pump = []
    B_Pump = []

    UL_A_T1 = 1100000
    UL_A_T3 = 1100000
    UL_A_Comp = 1100000
    UL_A_Pump = 1100000

    model.CCompressor2 = Var(Factories,initialize=1,within=NonNegativeReals,bounds=(0.000,UL_A_Comp**0.6038))
    model.CPump2 = Var(Factories, Demands, initialize=1,within=NonNegativeReals,bounds=(0.000,UL_A_Pump**0.8335))
    model.CPT1 = Var(Factories,Demands, initialize=1,within=NonNegativeReals,bounds=(0.000,math.e**(0.0787*UL_A_T1**0.5)))
    model.CPT3 = Var(Factories,Demands, initialize=1,within=NonNegativeReals,bounds=(0.000, UL_A_T3**0.5))

    for f in Factories:
        A_Comp.append(list(np.arange(0,1100000,5000)))
        B_Comp.append([])

        for i in range(len(A_Comp[f])):
            B_Comp[f].append(A_Comp[f][i]**0.6038)

        
        name_Comp = 'Constraint_comp'+str(f)
        model.add_component(name_Comp,Piecewise(model.CCompressor2[f],model.CCompressor[f],
                            pw_pts=A_Comp[f],
                            pw_constr_type='EQ',
                            f_rule=B_Comp[f],
                            pw_repn='SOS2'))
        
        A_Pump.append([])
        B_Pump.append([])
        A_CSA1.append([])
        B_T1.append([])
        A_CSA3.append([])
        B_T3.append([])
        
        for d in Demands:
            A_Pump[f].append(list(np.arange(0,1100000,5000)))
            B_Pump[f].append([])

            for i in range(len(A_Pump[f][d])):
                B_Pump[f][d].append(A_Pump[f][d][i]**0.8335)

            namepump = "constraint_pump_" + str(f)+ str(d)
            model.add_component(namepump,Piecewise( model.CPump2[f,d],model.CPump[f,d],
                                    pw_pts=A_Pump[f][d],
                                    pw_constr_type='EQ',
                                    f_rule=B_Pump[f][d],
                                    pw_repn='SOS2'))
            
            A_CSA1[f].append(list(np.arange(0,1100000,5000)))
            B_T1[f].append([])
            A_CSA3[f].append(list(np.arange(0,1100000,5000)))
            B_T3[f].append([])

            for i in range(len(A_CSA1[f][d])):
                B_T1[f][d].append(math.e**(0.0787*A_CSA1[f][d][i]**0.5))

            for i in range(len(A_CSA3[f][d])):   
                B_T3[f][d].append(A_CSA3[f][d][i]**0.5)

                
            namet1 = "constraint_t1"+str(f)+str(d)
            model.add_component(namet1,Piecewise(model.CPT1[f,d],model.CSA[f,d],
                                pw_pts=A_CSA1[f][d],
                                pw_constr_type='EQ',
                                f_rule=B_T1[f][d],
                                pw_repn='SOS2'))
            
            namet3 = "constraint_t3"+str(f)+str(d)
            model.add_component(namet3,Piecewise(model.CPT3[f,d],model.CSA[f,d],
                                pw_pts=A_CSA3[f][d],
                                pw_constr_type='EQ',
                                f_rule=B_T3[f][d],
                                pw_repn='SOS2'))

    ## Piecewise_Reapproximation
    def Piecewise_Reapproximate():
        for f in Factories:
                    
                A_Comp[f].insert(0,model.CCompressor[f]())
                A_Comp[f].sort()
                B_Comp[f].insert(A_Comp[f].index(model.CCompressor[f]()),model.CCompressor[f]()**0.6038)
                
                name_Comp = 'Constraint_comp'+str(f)
                model.del_component(name_Comp)
                model.add_component(name_Comp,Piecewise(model.CCompressor2[f],model.CCompressor[f],
                                pw_pts=A_Comp[f],
                                pw_constr_type='EQ',
                                f_rule=B_Comp[f],
                                pw_repn='SOS2'))
                
                        
                for d in Demands:
                    A_Pump[f][d].insert(0,model.CPump[f,d]())
                    A_Pump[f][d].sort()
                    B_Pump[f][d].insert(A_Pump[f][d].index(model.CPump[f,d]()),model.CPump[f,d]()**0.8335)
            
                    namepump = "constraint_pump_" + str(f)+ str(d)
                    model.del_component(namepump)
                    model.add_component(namepump, Piecewise(model.CPump2[f,d],model.CPump[f,d],
                                        pw_pts=A_Pump[f][d],
                                        pw_constr_type='EQ',
                                        f_rule=B_Pump[f][d],
                                        pw_repn='SOS2'))
                    
                    A_CSA1[f][d].insert(0,model.CSA[f,d]())
                    A_CSA1[f][d].sort()
                    B_T1[f][d].insert(A_CSA1[f][d].index(model.CSA[f,d]()),math.e**(0.0787*model.CSA[f,d]()**0.5))

                    namet1 = "constraint_t1"+str(f)+str(d)
                    model.del_component(namet1)
                    model.add_component(namet1,Piecewise(model.CPT1[f,d],model.CSA[f,d],
                                    pw_pts=A_CSA1[f][d],
                                    pw_constr_type='EQ',
                                    f_rule=B_T1[f][d],
                                    pw_repn='SOS2'))
                    
                    A_CSA3[f][d].insert(0,model.CSA[f,d]())
                    A_CSA3[f][d].sort()
                    B_T3[f][d].insert(A_CSA3[f][d].index(model.CSA[f,d]()),model.CSA[f,d]()**0.5)

                namet3 = "constraint_t3"+str(f)+str(d)
                model.del_component(namet3)
                model.add_component(namet3,Piecewise(model.CPT3[f,d],model.CSA[f,d],
                                pw_pts=A_CSA3[f][d],
                                pw_constr_type='EQ',
                                f_rule=B_T3[f][d],
                                pw_repn='SOS2'))
        

    ##OOBJ
    def oobj():
        val = sum((0
                +(PCWT + POWT)*model.NWT[f]() #Capex & Opex Costs of Wind Turbines
                +(PCDes + PODes)*model.CDes[f]() #Capex & Opex Costs of Desalination
                +(PCEL + POEL)*model.CEL[f]() #Capex & Opex Costs of Electrolyzer
                +sum((PCP1*math.e**(0.0787*model.CSA[f,d]()**(0.5)) +PCP2*model.CSA[f,d]()+PCP3*model.CSA[f,d]()**0.5+PCP4)*(RTT[f,d]*0.5) for d in Demands)*1.5*1.5 #Capex & Opex Costs of Pipelines
                +(PCS +POS)*model.Cstore[f]() #Capex & Opex Costs of Storage
                +(PCC +POC)*model.CCompressor[f]()**0.6038 #Capex & Opex Costs of Compression
                +sum((PCT +POT)*model.CPump[f,d]()**0.8335 for d in Demands) #Capex & Opex Costs of Pumping
                +FP_Cost*model.CEL[f]() #Capex & Opex Costs of Floating Platform
                ) for f in Factories)
        return val

    ##
    def objrule(model):
        return (sum(0
            +(PCWT + POWT)*model.NWT[f] #Capex & Opex Costs of Wind Turbines
            +(PCDes + PODes)*model.CDes[f] #Capex & Opex Costs of Desalination
            +(PCEL + POEL)*model.CEL[f] #Capex & Opex Costs of Electrolyzer
            +sum((PCP1*model.CPT1[f,d] +PCP2*model.CSA[f,d] +PCP3*model.CPT3[f,d]+PCP4)*(RTT[f,d]*1) for d in Demands)*1.5*1.5 #Capex & Opex Costs of Pipelines
            +(PCS +POS)*model.Cstore[f] #Capex & Opex Costs of Storage
            +(PCC +POC)*model.CCompressor2[f] #Capex & Opex Costs of Compression
            +sum((PCT +POT)*model.CPump2[f,d] for d in Demands) #Capex & Opex Costs of Pumping
            +FP_Cost*model.CEL[f]() #Capex & Opex Costs of Floating Platform
            for f in Factories))

    model.obj = Objective(rule=objrule, sense=minimize)

    def UB_Func():
        val = sum((0
                +(PCWT + POWT)*model.NWT[f]() #Capex & Opex Costs of Wind Turbines
                +(PCDes + PODes)*model.CDes[f]() #Capex & Opex Costs of Desalination
                +(PCEL + POEL)*model.CEL[f]() #Capex & Opex Costs of Electrolyzer
                +sum((PCP1*model.CPT1[f,d]() +PCP2*model.CSA[f,d]() +PCP3*model.CPT3[f,d]() +PCP4)*(RTT[f,d]*0.5) for d in Demands)*1.5*1.5
                +(PCS +POS)*model.Cstore[f]() #Capex & Opex Costs of Storage
                +(PCC +POC)*model.CCompressor[f]()**0.6038 #Capex & Opex Costs of Compression
                +sum((PCT +POT)*model.CPump[f,d]()**0.8335 for d in Demands) #Capex & Opex Costs of Pumping
                +FP_Cost*model.CEL[f]() #Capex & Opex Costs of Floating Platform
        )for f in Factories)
        return val

    def LB_Func():
        val = sum((0
                +(PCWT + POWT)*model.NWT[f]() #Capex & Opex Costs of Wind Turbines
                +(PCDes + PODes)*model.CDes[f]() #Capex & Opex Costs of Desalination
                +(PCEL + POEL)*model.CEL[f]() #Capex & Opex Costs of Electrolyzer
                +sum((PCP1*math.e**(0.0787*model.CSA[f,d]()**(0.5)) +PCP2*model.CSA[f,d]()+PCP3*model.CSA[f,d]()**0.5+PCP4)*(RTT[f,d]*0.5) for d in Demands)*1.5*1.5#Capex & Opex Costs of Pipelines
                +(PCS +POS)*model.Cstore[f]() #Capex & Opex Costs of Storage
                +(PCC +POC)*model.CCompressor2[f]() #Capex & Opex Costs of Compression
                +sum((PCT +POT)*model.CPump2[f,d]() for d in Demands) #Capex & Opex Costs of Pumping
                +FP_Cost*model.CEL[f]() #Capex & Opex Costs of Floating Platform
        )for f in Factories)
        return val
    
    ## Constraints

    # Facility Selection:
    BigM = 9999999
    model.FacilitySelection = ConstraintList()
    for f in Factories:
        model.FacilitySelection.add(model.NWT[f]<=BigM*model.selector[f])
            
        model.FacilitySelection.add(model.CDes[f]<=BigM*model.selector[f])
        model.FacilitySelection.add(model.CEL[f]<=BigM*model.selector[f])
        model.FacilitySelection.add(model.Cstore[f]<=BigM*model.selector[f])
        model.FacilitySelection.add(model.CCompressor[f]<=BigM*model.selector[f])
        for d in Demands:
            model.FacilitySelection.add(model.CSA[f,d]<=BigM*model.selector[f])
            model.FacilitySelection.add(model.CPump[f,d]<=BigM*model.selector[f])

    model.FacilitySelection.add(sum(model.selector[f] for f in Factories) == 1)

    # Energy Balance Constraints:
    model.energybalance = ConstraintList()
    for f in Factories:
        for t in Times:
            model.energybalance.add(model.EWT[f,t] == model.EDes[f,t] + model.EEL[f,t] + model.ECompressor[f,t] + sum(model.EPump[f,d,t] for d in Demands)) 

    model.windpower = ConstraintList()
    for f in Factories:
        for t in Times:
            model.windpower.add(model.EWT[f,t] == CF[f][t]*0.015*24*model.NWT[f]) 

    model.desalination = ConstraintList()
    for f in Factories:
        for t in Times:
            model.desalination.add(model.EDes[f,t] == 52.5*10**(-3)*model.F[f,t]) 

    model.electrolysis = ConstraintList()
    for f in Factories:
        for t in Times:
            model.electrolysis.add(model.EEL[f,t] == model.F[f,t]*50*10**(0)) 

    model.compression = ConstraintList()
    for f in Factories:
        for t in Times:
            model.compression.add(model.ECompressor[f,t] == (model.F[f,t])*33.3*10**(0)/(24.5))

    model.pumping = ConstraintList()
    for f in Factories:
        for d in Demands:
            for t in Times:
                model.pumping.add(model.EPump[f,d,t] == 2.7e-7*model.M[f,d,t]*(deltaP_l)*(RTT[f,d]*50)/Rho)

    # Capacity Constraints:

    model.c1 = ConstraintList()
    for f in Factories:
        for t in Times:
            model.c1.add(model.F[f,t]/24 <= model.CDes[f]) #Desalination Capacity

    model.c2 = ConstraintList()
    for f in Factories:
        for t in Times:
            model.c2.add(model.EEL[f,t]/24 <= model.CEL[f]) #Electrolyzer Capacity
            
    model.c3 = ConstraintList()
    for f in Factories:
        for t in Times:
            model.c3.add(model.ECompressor[f,t]/24 <= model.CCompressor[f]) #Compressor Capacity

    model.c4 = ConstraintList()
    for f in Factories:
        for d in Demands:
            for t in Times:
                model.c4.add(model.EPump[f,d,t]/24 <= model.CPump[f,d]) #Pump Capacity
    
    #Transport & Desmand Constraints:
    
    model.mb = ConstraintList()
    for f in Factories:
        for t in Times:
            if t == 0:
                model.mb.add(model.Lstore[f,t] == (model.Lstore[f,364]+model.F[f,t]-sum(model.M[f,d,t] for d in Demands)))
            else:
                model.mb.add(model.Lstore[f,t] == (model.Lstore[f,t-1]+model.F[f,t]-sum(model.M[f,d,t] for d in Demands)))
                
    model.v2 = ConstraintList()
    for f in Factories:
        for t in Times:
            model.v2.add(model.Lstore[f,t]<=model.Cstore[f])

    model.v3 = ConstraintList()
    for f in Factories:
        for d in Demands:
            for t in Times:
                model.v3.add(model.M[f,d,t]/24 <= model.CSA[f,d]*V_max*Rho*(2.34e-6)) #conversion factor from kg/s to kt/hr

    model.demand = ConstraintList()
    for d in Demands:
        model.demand.add(sum(model.M[f,d,t] for f in Factories for t in Times) >= DemandValue[d])

    #Solve
    tic = time.perf_counter()

    q1=[]
    ub1=[]
    lb1=[]
    gap1=[]
    #change2 end

    solver = SolverFactory('gurobi')
    solver.options['mipgap'] = 0.01
    solver.options['MIPFocus'] = 1
    solver.options['Heuristics'] = 1
    solver.options['cuts']=3

    solver.options['ScaleFlag']=2

    #mip_start = warmstarter()
    solver.solve(model,tee=True)

    Gap_Convex = 1-(abs(oobj())/abs(UB_Func()))
    Gap_Concave = 1-(abs(LB_Func())/abs(oobj()))

    print("Gap_Convex=",Gap_Convex)
    print("Gap_Concave=",Gap_Concave)

    while (Gap_Convex>0.00001 or Gap_Concave>0.00001):

        Piecewise_Reapproximate()
        
        model.del_component('model.obj')
        model.obj = Objective(rule=objrule, sense=minimize)
        
        #mip_start = warmstarter()
        #solver.solve(model,warmstart=mip_start,tee=True)
        solver.solve(model,tee=True)

        Gap_Convex = 1-(abs(oobj())/abs(UB_Func()))
        Gap_Concave = 1-(abs(LB_Func())/abs(oobj()))

        print("Gap_Convex=",Gap_Convex)
        print("Gap_Concave=",Gap_Concave)
    
    toc = time.perf_counter()
    print(f"Completed in {toc - tic:0.4f} seconds")
    
    # Loop through each variable in the model
    full_model_data = {'Model':'Northeast'}
    
    for var in model.component_data_objects(Var, active=True):
        full_model_data[var.name] = var.value
    
    
    def common_costs_Eval(d):
        val = sum((0
                +(PCWT + POWT)*model.NWT[f]() #Capex & Opex Costs of Wind Turbines
                +(PCDes + PODes)*model.CDes[f]() #Capex & Opex Costs of Desalination
                +(PCEL + POEL)*model.CEL[f]() #Capex & Opex Costs of Electrolyzer
                +(PCS +POS)*model.Cstore[f]() #Capex & Opex Costs of Storage
                +(PCC +POC)*model.CCompressor[f]()**0.6038 #Capex & Opex Costs of Compression
                +FP_Cost*model.CEL[f]() #Capex & Opex Costs of Floating Platform
                ) for f in Factories)
        return val

    def sp_costs_Eval(d):
        ReVal = 0
        for f in Factories:
            if model.selector[f]() == 1:
                val = (0
                +((PCP1*math.e**(0.0787*model.CSA[f,d]()**(0.5)) +PCP2*model.CSA[f,d]()+PCP3*model.CSA[f,d]()**0.5+PCP4)*(RTT[f,d]*0.5))*1.5*1.5 #Capex & Opex Costs of Pipelines
                +(PCT +POT)*model.CPump[f,d]()**0.8335 #Capex & Opex Costs of Pumping
                )
                ReVal = ReVal + val
        return ReVal

    state_results = pd.DataFrame()

    for d in Demands:
        model_data = {'Model': hub}
        model_data['State'] = states[d]
        model_data['Objective'] = model.obj()
        

        model_data['Cost of Wind Turbines'] = sum((PCWT + POWT)*model.NWT[f]() for f in Factories)
        model_data['Cost of Desalination'] = sum((PCDes + PODes)*model.CDes[f]() for f in Factories)
        model_data['Cost of Electrolyzer'] = sum((PCEL + POEL)*model.CEL[f]() for f in Factories)
       
        model_data['Cost of Compression'] = sum((PCC +POC)*model.CCompressor[f]()**0.6038 for f in Factories)
        model_data['Cost of Storage'] = sum((PCS +POS)*model.Cstore[f]() for f in Factories)
        
        model_data['cost of floating platform'] = FP_Cost*sum(model.CEL[f]() for f in Factories)

        model_data['Common Costs'] = common_costs_Eval(d)
        model_data['Pipeline Costs'] = sp_costs_Eval(d)
        model_data['H2 Shipped'] = sum(model.M[f,d,t]() for f in Factories for t in Times)
        model_data['H2 Produced'] = sum(model.F[f,t]() for f in Factories for t in Times)
        model_data['Common_LCOH'] = common_costs_Eval(d)/sum(model.F[f,t]() for f in Factories for t in Times)
        model_data['Pipelines_LCOH'] = sp_costs_Eval(d)/sum(model.M[f,d,t]() for f in Factories for t in Times)
        model_data['Total_LCOH'] = model_data['Common_LCOH']+model_data['Pipelines_LCOH']

        model_data['Energy Produced'] = sum(model.EWT[f,t]() for f in Factories for t in Times)
        model_data['Energy for Desalination'] = sum(model.EDes[f,t]() for f in Factories for t in Times)
        model_data['Energy for Electrolysis'] = sum(model.EEL[f,t]() for f in Factories for t in Times)
        model_data['Energy for Compression'] = sum(model.ECompressor[f,t]() for f in Factories for t in Times)
        model_data['Energy for Pumping'] = sum(model.EPump[f,d,t]() for f in Factories for d in Demands for t in Times)
        
        model_data['GHG of Wind Turbines'] = sum(model.NWT[f]() for f in Factories)*15/11*1.65e7
        model_data['GHG of Desalination'] = model_data['H2 Produced']*(1000/2*18*20)*(2.25+4.85E-1)
        model_data['GHG of Electrolyzer'] = sum(model.CEL[f]() for f in Factories)*1000*5.85e4
        model_data['GHG of pipeline'] = (4*sum(model.CSA[f,d]() for f in Factories)/3.14)**0.5*2.54*10*(100/1000)*1.5*1.97E6
        model_data['GHG of storage'] = sum(model.Cstore[f]() for f in Factories)*1000*1.35e5
        model_data['GHG of compression'] = sum(model.CCompressor[f]() for f in Factories)*1000/32*8.71e05
        model_data['GHG of pumping'] = sum(model.CPump[f,d]() for f in Factories)*1000/32*8.71e05
        model_data['GHG of Galvalume'] = 3.14/4*((4*sum(model.CSA[f,d]() for f in Factories)/3.14)**0.5*2.54*10*0.001)**2*100*1000*5.55
    
        state_results = state_results.append(model_data, ignore_index=True)

    return full_model_data, state_results
    
            

In [ ]:
df = pd.read_excel('Hubwise_Casestudy_guide.xlsx')

#Get Hubs
hubs = df['Hub'].tolist()

#get states
states = df['states'].tolist()
#convert comma separated string to list
states = [i.split(',') for i in states]

#Get Buoys
buoys = df['buoys'].tolist()
#convert comma separated string to list
buoys = [i.split(',') for i in buoys]

#Get Factories
Factories = df['Factories'].tolist()
#convert comma separated string to list
Factories = [i.split(',') for i in Factories]

#convert str numbers to int
for i in range(len(Factories)):
    Factories[i] = [int(j) for j in Factories[i]]

#Get Demands
Demands = df['Demands'].tolist()
#convert comma separated string to list
Demands = [i.split(',') for i in Demands]

#convert str numbers to int
for i in range(len(Demands)):
    Demands[i] = [int(j) for j in Demands[i]]

#Get DemandValues
DemandValue = df['DemandValue'].tolist()
#convert comma separated string to list
DemandValue = [i.split(',') for i in DemandValue]

#convert str numbers to float
for i in range(len(DemandValue)):
    DemandValue[i] = [float(j) for j in DemandValue[i]]

#Get RTT filenames
RTTFile = df['RTTFile'].tolist()

full_states_results = pd.DataFrame()

for i in range(len(hubs)):
    full_model_data, state_results = Optimal_Design(hubs[i],states[i],buoys[i],Factories[i],Demands[i],DemandValue[i],RTTFile[i])
    
    full_model_data = pd.DataFrame.from_dict(full_model_data, orient='index')
    full_model_data = full_model_data.transpose()
    full_model_data.to_csv('Fullmodel_results_Hubwise_CGH2' +str(hubs[i])+'hub (2).csv')
    
    full_states_results = full_states_results.append(state_results, ignore_index=True)

full_states_results.to_excel('Hubwise_CGH2_final_heatmap_results.xlsx')